In [7]:
!pip install evaluate nltk bert_score

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install nltk -U
!pip install evaluate -U

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [15]:
!pip install --upgrade regex
!pip install --upgrade evaluate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [20]:
!pip install evaluate

Defaulting to user installation because normal site-packages is not writeable


In [33]:
!pip install -U nltk

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.5 MB 3.9 MB/s eta 0:00:01


In [1]:
import os
import regex as re
import csv
import pandas as pd
from evaluate import load

meteor = load('meteor')
bertscore = load("bertscore")

post_processed_results = pd.read_csv("/work/AI-EHR/discharge_me/data/Experiments/zeroshot-llama8b_instruct/troy_scoring_experiment/post_processed_results.csv")
#post_processed_results

discharge_target = pd.read_csv("/work/AI-EHR/discharge_me/data/Experiments/zeroshot-llama8b_instruct/troy_scoring_experiment/discharge_target.csv")
#discharge_target

hadm_id_for_discharge_me_scoring = pd.read_csv("/work/AI-EHR/discharge_me/data/Experiments/zeroshot-llama8b_instruct/troy_scoring_experiment/hadm_id_for_discharge_me_scoring.csv")
#hadm_id_for_discharge_me_scoring

[nltk_data] Downloading package wordnet to /home/tr.caron/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/tr.caron/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/tr.caron/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
BHC_scores_df = pd.DataFrame()
DI_scores_df = pd.DataFrame()

# Merge the post_processed_results dataframe with the hadm_id_for_discharge_me_scoring dataframe
merged_df1 = pd.merge(hadm_id_for_discharge_me_scoring, post_processed_results, on='hadm_id', how='inner')

# Renaming columns for clarity
merged_df1 = merged_df1.rename(columns={
    'brief_hospital_course': 'BHC_LLM_out',
    'discharge_instructions': 'DI_LLM_out'
})

# Merge the post_processed_results dataframe with the hadm_id_for_discharge_me_scoring dataframe
merged_df2 = pd.merge(merged_df1, discharge_target, on='hadm_id', how='inner')

# Renaming columns for clarity
merged_df2 = merged_df2.rename(columns={
    'brief_hospital_course': 'BHC_human_generated',
    'discharge_instructions': 'DI_human_generated'
})

# BHC DATAFRAME

# Copy relevant columns from the original DataFrame
BHC_scores_df['hadm_id'] = merged_df2['hadm_id']
BHC_scores_df['BHC_LLM_out'] = merged_df2['BHC_LLM_out']
BHC_scores_df['BHC_human_generated'] = merged_df2['BHC_human_generated']

# Initialize the 'meteor_score' column with zeros
BHC_scores_df['meteor_score'] = 0.0
BHC_scores_df['bertscore_precision'] = 0.0
BHC_scores_df['bertscore_recall'] = 0.0
BHC_scores_df['bertscore_f1'] = 0.0

# Calculate METEOR score for each row
for index, row in BHC_scores_df.iterrows():
    prediction = [row['BHC_LLM_out']]
    reference = [row['BHC_human_generated']]
    result = meteor.compute(predictions=prediction, references=reference)
    BHC_scores_df.at[index, 'meteor_score'] = result['meteor']
    bertscore_result = bertscore.compute(predictions=prediction, references=reference, model_type="distilbert-base-uncased")
    BHC_scores_df.at[index, 'bertscore_precision'] = bertscore_result['precision'][0]
    BHC_scores_df.at[index, 'bertscore_recall'] = bertscore_result['recall'][0]
    BHC_scores_df.at[index, 'bertscore_f1'] = bertscore_result['f1'][0]

    
# DI DATAFRAME

# Copy relevant columns from the original DataFrame
DI_scores_df['hadm_id'] = merged_df2['hadm_id']
DI_scores_df['DI_LLM_out'] = merged_df2['DI_LLM_out']
DI_scores_df['DI_human_generated'] = merged_df2['DI_human_generated']

# Initialize the 'meteor_score' column with zeros
DI_scores_df['meteor_score'] = 0.0
DI_scores_df['bertscore_precision'] = 0.0
DI_scores_df['bertscore_recall'] = 0.0
DI_scores_df['bertscore_f1'] = 0.0

# Calculate METEOR score for each row
for index, row in DI_scores_df.iterrows():
    prediction = [row['DI_LLM_out']]
    reference = [row['DI_human_generated']]
    result = meteor.compute(predictions=prediction, references=reference)
    DI_scores_df.at[index, 'meteor_score'] = result['meteor']
    bertscore_result = bertscore.compute(predictions=prediction, references=reference, model_type="distilbert-base-uncased")
    DI_scores_df.at[index, 'bertscore_precision'] = bertscore_result['precision'][0]
    DI_scores_df.at[index, 'bertscore_recall'] = bertscore_result['recall'][0]
    DI_scores_df.at[index, 'bertscore_f1'] = bertscore_result['f1'][0]

In [22]:
BHC_scores_df.to_csv("BHC_scores_output.csv",index=False)
DI_scores_df.to_csv("DI_scores_output.csv",index=False)

In [18]:
BHC_scores_df.describe()

,hadm_id,meteor_score,bertscore_precision,bertscore_recall,bertscore_f1
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000
mean,2.498990e+07,0.228323,0.781773,0.775893,0.778593
std,2.727081e+06,0.056019,0.027242,0.026346,0.023233
min,2.006350e+07,0.089823,0.694748,0.692853,0.699865
25%,2.271984e+07,0.190075,0.764479,0.762092,0.765064
50%,2.482564e+07,0.223672,0.781092,0.775661,0.778624
75%,2.711815e+07,0.258857,0.801148,0.792880,0.795442
max,2.999192e+07,0.392270,0.843972,0.856289,0.842104


In [19]:
DI_scores_df.describe()

,hadm_id,meteor_score,bertscore_precision,bertscore_recall,bertscore_f1
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000
mean,2.498990e+07,0.286782,0.767692,0.809610,0.787790
std,2.727081e+06,0.064048,0.024551,0.032874,0.024337
min,2.006350e+07,0.057545,0.624597,0.685145,0.656671
25%,2.271984e+07,0.246495,0.751588,0.796135,0.775745
50%,2.482564e+07,0.284390,0.768425,0.814090,0.792635
75%,2.711815e+07,0.328802,0.785299,0.831387,0.804345
max,2.999192e+07,0.455503,0.838272,0.869256,0.847488


# LLama 3.1 Scoring

In [3]:
llama31_processed_results = pd.read_csv("/work/AI-EHR/discharge_me/data/Experiments/zeroshot-llama8b_instruct/troy_scoring_experiment/llama3.1_processed_results.csv")
llama31_processed_results

,filename,letter,bhc_summary
0,hadm_id_25779173_inputs.txt_masked.txt,"Dear [Patient's Name],\n\nI hope this letter f...",**Summary of Hospital Visit**\n\nThe patient h...
1,hadm_id_20622567_inputs.txt_masked.txt,"Dear [Patient],\n\nI hope this letter finds yo...",The patient has a complex past medical history...
2,hadm_id_25086586_inputs.txt_masked.txt,"Dear [Patient's Name],\n\nI hope this letter f...",**Summary Sentence:**\nA 45-year-old female wi...
3,hadm_id_24303484_inputs.txt_masked.txt,"Dear [Patient's Name],\n\nI hope this letter f...",**Summary Sentence:** A 55-year-old female wit...
4,hadm_id_25567083_inputs.txt_masked.txt,"Dear [Patient's Name],\n\nI hope this letter f...",**Summary Sentence:** A 67-year-old patient wi...
...,...,...,...
245,hadm_id_20593906_inputs.txt_masked.txt,"Dear [Patient's Name],\n\nI hope this letter f...",**Summary Sentence:** A patient with a history...
246,hadm_id_24717214_inputs.txt_masked.txt,"Dear [Patient's Name],\n\nI hope this letter f...",Here is a summary of the patient's hospital vi...
247,hadm_id_28178093_inputs.txt_masked.txt,"Dear [Patient's Name],\n\nI hope this letter f...",The patient is a [age]-year-old with a history...
248,hadm_id_24982158_inputs.txt_masked.txt,"Dear [Patient's Name],\n\nI hope this letter f...",Here is a summary of the patient's hospital vi...


In [4]:
# Extracting hadm_ids from filename column
def extract_hadm_id(filename):
    match = re.search(r'\d+', filename)
    return match.group(0) if match else filename

llama31_processed_results['filename'] = llama31_processed_results['filename'].apply(extract_hadm_id)

In [5]:
# Renaming columns
llama31_processed_results = llama31_processed_results.rename(columns={
    'filename': 'hadm_id',
    'letter': 'DI_llama3.1_out',
    'bhc_summary': 'BHC_llama3.1_out'
})

llama31_processed_results['hadm_id'] = llama31_processed_results['hadm_id'].astype(int)

llama31_processed_results

,hadm_id,DI_llama3.1_out,BHC_llama3.1_out
0,25779173,"Dear [Patient's Name],\n\nI hope this letter f...",**Summary of Hospital Visit**\n\nThe patient h...
1,20622567,"Dear [Patient],\n\nI hope this letter finds yo...",The patient has a complex past medical history...
2,25086586,"Dear [Patient's Name],\n\nI hope this letter f...",**Summary Sentence:**\nA 45-year-old female wi...
3,24303484,"Dear [Patient's Name],\n\nI hope this letter f...",**Summary Sentence:** A 55-year-old female wit...
4,25567083,"Dear [Patient's Name],\n\nI hope this letter f...",**Summary Sentence:** A 67-year-old patient wi...
...,...,...,...
245,20593906,"Dear [Patient's Name],\n\nI hope this letter f...",**Summary Sentence:** A patient with a history...
246,24717214,"Dear [Patient's Name],\n\nI hope this letter f...",Here is a summary of the patient's hospital vi...
247,28178093,"Dear [Patient's Name],\n\nI hope this letter f...",The patient is a [age]-year-old with a history...
248,24982158,"Dear [Patient's Name],\n\nI hope this letter f...",Here is a summary of the patient's hospital vi...


In [6]:
Llama31_BHC_scores_df = pd.DataFrame()
LLama31_DI_scores_df = pd.DataFrame()

# Merge the post_processed_results dataframe with the hadm_id_for_discharge_me_scoring dataframe
merged_df = pd.merge(llama31_processed_results, discharge_target, on='hadm_id', how='inner')

# Renaming columns for clarity
merged_df = merged_df.rename(columns={
    'brief_hospital_course': 'BHC_human_generated',
    'discharge_instructions': 'DI_human_generated'
})

# BHC DATAFRAME

# Copy relevant columns from the original DataFrame
Llama31_BHC_scores_df['hadm_id'] = merged_df['hadm_id']
Llama31_BHC_scores_df['BHC_llama3.1_out'] = merged_df['BHC_llama3.1_out']
Llama31_BHC_scores_df['BHC_human_generated'] = merged_df['BHC_human_generated']

# Initialize the 'meteor_score' column with zeros
Llama31_BHC_scores_df['meteor_score'] = 0.0
Llama31_BHC_scores_df['bertscore_precision'] = 0.0
Llama31_BHC_scores_df['bertscore_recall'] = 0.0
Llama31_BHC_scores_df['bertscore_f1'] = 0.0

# Calculate METEOR score for each row
for index, row in Llama31_BHC_scores_df.iterrows():
    prediction = [row['BHC_llama3.1_out']]
    reference = [row['BHC_human_generated']]
    result = meteor.compute(predictions=prediction, references=reference)
    Llama31_BHC_scores_df.at[index, 'meteor_score'] = result['meteor']
    bertscore_result = bertscore.compute(predictions=prediction, references=reference, model_type="distilbert-base-uncased")
    Llama31_BHC_scores_df.at[index, 'bertscore_precision'] = bertscore_result['precision'][0]
    Llama31_BHC_scores_df.at[index, 'bertscore_recall'] = bertscore_result['recall'][0]
    Llama31_BHC_scores_df.at[index, 'bertscore_f1'] = bertscore_result['f1'][0]

    
# DI DATAFRAME

# Copy relevant columns from the original DataFrame
LLama31_DI_scores_df['hadm_id'] = merged_df['hadm_id']
LLama31_DI_scores_df['DI_llama3.1_out'] = merged_df['DI_llama3.1_out']
LLama31_DI_scores_df['DI_human_generated'] = merged_df['DI_human_generated']

# Initialize the 'meteor_score' column with zeros
LLama31_DI_scores_df['meteor_score'] = 0.0
LLama31_DI_scores_df['bertscore_precision'] = 0.0
LLama31_DI_scores_df['bertscore_recall'] = 0.0
LLama31_DI_scores_df['bertscore_f1'] = 0.0

# Calculate METEOR score for each row
for index, row in LLama31_DI_scores_df.iterrows():
    prediction = [row['DI_llama3.1_out']]
    reference = [row['DI_human_generated']]
    result = meteor.compute(predictions=prediction, references=reference)
    LLama31_DI_scores_df.at[index, 'meteor_score'] = result['meteor']
    bertscore_result = bertscore.compute(predictions=prediction, references=reference, model_type="distilbert-base-uncased")
    LLama31_DI_scores_df.at[index, 'bertscore_precision'] = bertscore_result['precision'][0]
    LLama31_DI_scores_df.at[index, 'bertscore_recall'] = bertscore_result['recall'][0]
    LLama31_DI_scores_df.at[index, 'bertscore_f1'] = bertscore_result['f1'][0]

In [7]:
Llama31_BHC_scores_df.to_csv("Llama31_BHC_scores_df.csv",index=False)
LLama31_DI_scores_df.to_csv("LLama31_DI_scores_df.csv",index=False)

In [34]:
Llama31_BHC_scores_df.describe()

,hadm_id,meteor_score,bertscore_precision,bertscore_recall,bertscore_f1
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000
mean,2.498990e+07,0.157368,0.690459,0.740717,0.714504
std,2.727081e+06,0.026986,0.020463,0.015798,0.014145
min,2.006350e+07,0.090215,0.613765,0.674191,0.647836
25%,2.271984e+07,0.138106,0.677188,0.730379,0.704674
50%,2.482564e+07,0.156987,0.691554,0.741176,0.715023
75%,2.711815e+07,0.175643,0.704102,0.750425,0.724642
max,2.999192e+07,0.246517,0.739935,0.780000,0.749060


In [35]:
LLama31_DI_scores_df.describe()

,hadm_id,meteor_score,bertscore_precision,bertscore_recall,bertscore_f1
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000
mean,2.498990e+07,0.249294,0.731451,0.766314,0.748185
std,2.727081e+06,0.039583,0.020221,0.032163,0.022494
min,2.006350e+07,0.054317,0.598936,0.572733,0.585542
25%,2.271984e+07,0.225616,0.719427,0.754271,0.738870
50%,2.482564e+07,0.249278,0.733091,0.771067,0.751188
75%,2.711815e+07,0.272244,0.745240,0.786181,0.762037
max,2.999192e+07,0.372774,0.781968,0.824977,0.802003


# Exploring their bertscore calculation + other scoring metrics used by task organizers

In [20]:
!pip install alignscore

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement alignscore
ERROR: No matching distribution found for alignscore


In [1]:
from bertscore import BertScore
# from align import AlignScorer
from bleu import Bleu
from rouge import Rouge, Rouge1, Rouge2, RougeL


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/shared/centos7/anaconda3/2022.05/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/shared/centos7/anaconda3/2022.05/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in la

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [30]:
bert_scorer = BertScore()
#align_scorer = AlignScorer()
bleu_scorer = Bleu(n=4)
rouge1_scorer = Rouge1()
rouge2_scorer = Rouge2()
rougeL_scorer = RougeL()

## Zero shot scoring

### BHC

In [17]:
refs = BHC_scores_df['BHC_human_generated'].tolist()
hyps = BHC_scores_df['BHC_LLM_out'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

BHC_scores_df['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
BHC_scores_df['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
BHC_scores_df['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
BHC_scores_df['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
BHC_scores_df['task-organizers-bertscores'] = bertscores

NameError: name 'BHC_scores_df' is not defined

In [12]:
BHC_scores_df.describe()

,hadm_id,meteor_score,bertscore_precision,bertscore_recall,bertscore_f1,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000,2.500000e+02,250.000000,250.000000,250.000000,250.000000
mean,2.498990e+07,0.228323,0.781773,0.775893,0.778593,3.292901e-02,0.313158,0.084421,0.187212,0.280098
std,2.727081e+06,0.056019,0.027242,0.026346,0.023233,4.418558e-02,0.068705,0.048726,0.049383,0.064906
min,2.006350e+07,0.089823,0.694748,0.692853,0.699865,1.881899e-14,0.103560,0.000000,0.064171,0.093414
25%,2.271984e+07,0.190075,0.764479,0.762092,0.765064,8.316304e-10,0.267873,0.049844,0.151071,0.238340
50%,2.482564e+07,0.223672,0.781092,0.775661,0.778624,1.016631e-02,0.317325,0.080123,0.184710,0.274941
75%,2.711815e+07,0.258857,0.801148,0.792880,0.795442,5.238075e-02,0.353456,0.106464,0.215281,0.325899
max,2.999192e+07,0.392270,0.843972,0.856289,0.842104,2.477786e-01,0.478964,0.273171,0.357488,0.454456


In [27]:
BHC_scores_df.to_csv("Zeroshot_BHC_scores_df.csv",index=False)

In [33]:
BHC_scores_df.describe()

,hadm_id,meteor_score,bertscore_precision,bertscore_recall,bertscore_f1,task-organizers-bertscores
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000,250.000000
mean,2.498990e+07,0.228323,0.781773,0.775893,0.778593,0.280098
std,2.727081e+06,0.056019,0.027242,0.026346,0.023233,0.064906
min,2.006350e+07,0.089823,0.694748,0.692853,0.699865,0.093414
25%,2.271984e+07,0.190075,0.764479,0.762092,0.765064,0.238340
50%,2.482564e+07,0.223672,0.781092,0.775661,0.778624,0.274941
75%,2.711815e+07,0.258857,0.801148,0.792880,0.795442,0.325899
max,2.999192e+07,0.392270,0.843972,0.856289,0.842104,0.454456


### DI

In [13]:
refs = DI_scores_df['DI_human_generated'].tolist()
hyps = DI_scores_df['DI_LLM_out'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

DI_scores_df['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
DI_scores_df['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
DI_scores_df['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
DI_scores_df['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
DI_scores_df['task-organizers-bertscores'] = bertscores

In [26]:
DI_scores_df.to_csv("Zeroshot_DI_scores_df.csv",index=False)

In [14]:
DI_scores_df.describe()

,hadm_id,meteor_score,bertscore_precision,bertscore_recall,bertscore_f1,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000,2.500000e+02,250.000000,250.000000,250.000000,250.000000
mean,2.498990e+07,0.286782,0.767692,0.809610,0.787790,3.892457e-02,0.335732,0.091040,0.193016,0.297761
std,2.727081e+06,0.064048,0.024551,0.032874,0.024337,3.179452e-02,0.073329,0.043831,0.048429,0.067348
min,2.006350e+07,0.057545,0.624597,0.685145,0.656671,3.805116e-14,0.032154,0.000000,0.025723,-0.012774
25%,2.271984e+07,0.246495,0.751588,0.796135,0.775745,4.730563e-06,0.287983,0.060047,0.157283,0.262692
50%,2.482564e+07,0.284390,0.768425,0.814090,0.792635,4.189995e-02,0.344131,0.090899,0.196658,0.306672
75%,2.711815e+07,0.328802,0.785299,0.831387,0.804345,5.914508e-02,0.391819,0.121944,0.224890,0.339984
max,2.999192e+07,0.455503,0.838272,0.869256,0.847488,1.258121e-01,0.485246,0.223214,0.315271,0.491387


## LLama 3.1

### BHC

In [17]:
refs = Llama31_BHC_scores_df['BHC_human_generated'].tolist()
hyps = Llama31_BHC_scores_df['BHC_llama3.1_out'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

Llama31_BHC_scores_df['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
Llama31_BHC_scores_df['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
Llama31_BHC_scores_df['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
Llama31_BHC_scores_df['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
Llama31_BHC_scores_df['task-organizers-bertscores'] = bertscores

In [25]:
Llama31_BHC_scores_df.to_csv("Llama31_BHC_scores_df.csv",index=False)

In [19]:
Llama31_BHC_scores_df.describe()

,hadm_id,meteor_score,bertscore_precision,bertscore_recall,bertscore_f1,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000,2.500000e+02,250.000000,250.000000,250.000000,250.000000
mean,2.498990e+07,0.157368,0.690459,0.740717,0.714504,2.139893e-03,0.152039,0.018556,0.087473,0.099756
std,2.727081e+06,0.026986,0.020463,0.015798,0.014145,5.117766e-03,0.033425,0.011320,0.017417,0.037145
min,2.006350e+07,0.090215,0.613765,0.674191,0.647836,2.365986e-14,0.066955,0.000000,0.041037,-0.076516
25%,2.271984e+07,0.138106,0.677189,0.730379,0.704674,1.865113e-10,0.127510,0.010204,0.074766,0.075943
50%,2.482564e+07,0.156987,0.691554,0.741176,0.715023,2.650308e-10,0.149461,0.016282,0.086366,0.104037
75%,2.711815e+07,0.175643,0.704102,0.750425,0.724642,1.475232e-06,0.175283,0.024156,0.099416,0.124182
max,2.999192e+07,0.246517,0.739935,0.780000,0.749060,2.396370e-02,0.256532,0.054054,0.138801,0.200600


### DI

In [21]:
refs = LLama31_DI_scores_df['DI_human_generated'].tolist()
hyps = LLama31_DI_scores_df['DI_llama3.1_out'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

LLama31_DI_scores_df['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
LLama31_DI_scores_df['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
LLama31_DI_scores_df['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
LLama31_DI_scores_df['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
LLama31_DI_scores_df['task-organizers-bertscores'] = bertscores

In [24]:
LLama31_DI_scores_df.to_csv("LLama31_DI_scores_df.csv",index=False)

In [23]:
LLama31_DI_scores_df.describe()

,hadm_id,meteor_score,bertscore_precision,bertscore_recall,bertscore_f1,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000,2.500000e+02,250.000000,250.000000,250.000000,250.000000
mean,2.498990e+07,0.249294,0.731451,0.766314,0.748185,1.057583e-02,0.214340,0.045478,0.123986,0.199534
std,2.727081e+06,0.039583,0.020221,0.032163,0.022494,1.101298e-02,0.055918,0.019532,0.026778,0.057081
min,2.006350e+07,0.054317,0.598936,0.572733,0.585542,2.230784e-14,0.025157,0.000000,0.020964,-0.159655
25%,2.271984e+07,0.225616,0.719427,0.754271,0.738870,1.765776e-06,0.172448,0.030608,0.105978,0.172354
50%,2.482564e+07,0.249278,0.733091,0.771067,0.751188,1.128742e-02,0.212498,0.044499,0.122819,0.207927
75%,2.711815e+07,0.272244,0.745240,0.786181,0.762037,1.819708e-02,0.254366,0.056577,0.142208,0.236730
max,2.999192e+07,0.372774,0.781968,0.824977,0.802003,4.385450e-02,0.350305,0.110429,0.195519,0.327914


# Exploring different preprocessing approaches

In [15]:
pip install --upgrade numpy scipy

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
     |████████████████████████████████| 38.6 MB 4.0 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.0
    Uninstalling numpy-2.0.0:
      Successfully uninstalled numpy-2.0.0
ERROR: Could not install packages due to an OSError: [Errno 16] Device or resource busy: '.nfs5edcaef89eac6f6c000167ab'

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install numpy==1.24.0

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 17.3 MB 3.8 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.0
    Uninstalling numpy-2.0.0:
      Successfully uninstalled numpy-2.0.0
ERROR: Could not install packages due to an OSError: [Errno 16] Device or resource busy: '.nfs853ba563e58d6cb50000074e'

Note: you may need to restart the kernel to use updated packages.


In [30]:
import os
import regex as re
import csv
import pandas as pd
from evaluate import load

meteor = load('meteor')
bertscore = load("bertscore")

[nltk_data] Downloading package wordnet to /home/tr.caron/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/tr.caron/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/tr.caron/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
from evaluate import load

In [2]:
# Removes hashtags from a specified column in a specified dataframe
def remove_hashtags(df, column_name):
    # Check if the column exists in the dataframe
    if column_name in df.columns:
        # Remove all hashtags from the specified column
        df[column_name] = df[column_name].str.replace('#', '', regex=False)
    else:
        print(f"Column '{column_name}' does not exist in the dataframe.")
        
# Removes semicolons from a specified column in a specified dataframe
def remove_semicolons(df, column_name):
    # Check if the column exists in the dataframe
    if column_name in df.columns:
        # Remove all semicolons from the specified column
        df[column_name] = df[column_name].str.replace(';', '', regex=False)
    else:
        print(f"Column '{column_name}' does not exist in the dataframe.")

# Removes consecutive equal signs from a specified column in a specified dataframe
def remove_multiple_equals(df, column_name):
    # Check if the column exists in the dataframe
    if column_name in df.columns:
        # Find and replace multiple consecutive equal signs
        df[column_name] = df[column_name].str.replace(r'={2,}', '', regex=True)
    else:
        print(f"Column '{column_name}' does not exist in the dataframe.")

In [45]:
from bs4 import BeautifulSoup
import requests

# Scraping acronym list from webpage
# url = 'https://aspe.hhs.gov/common-acronyms'
# table = pd.read_html(url)

# table
url = 'https://www.asha.org/practice-portal/professional-issues/documentation-in-health-care/common-medical-abbreviations/'
headers = {'User-Agent': 'Mozilla/5.0'}

# Make the request with a user-agent
response = requests.get(url, headers=headers)

# Parse the page content
soup = BeautifulSoup(response.content, 'html.parser')

# Extract tables using pandas
dfs = pd.read_html(str(soup))

# Get the first table (adjust index if needed)
acronyms_df = dfs[0]

# Convert to a DataFrame and rename columns
acronyms_df = acronyms_df.rename(columns={0: "Acronyms", 1: "Meaning"})

# Display the DataFrame
acronyms_df

,Acronyms,Meaning
0,A,A
1,A.A.R.O.M.,active assistive range of motion
2,AAC,augmentative and alternative communication
3,A.B.G,arterial blood gas
4,a.c.,before meals
...,...,...
437,X,X
438,x,times
439,Y,Y
440,y.o.,years old


In [74]:
# Function to replace acronyms with their meanings
# def replace_acronyms(df, column_name, acronyms_df):
#     # Create a dictionary from the acronyms dataframe
#     acronyms_dict = pd.Series(acronyms_df['Meaning'].values, index=acronyms_df['Acronyms']).to_dict()

#     # replace acronyms in a string
#     def replace_in_string(text):
#         words = text.split()
#         replaced_text = ' '.join([acronyms_dict.get(word, word) for word in words])
#         return replaced_text

#     # Apply the function to the specified column
#     df[column_name] = df[column_name].apply(replace_in_string)
def replace_acronyms(df, column_name, acronyms_df):
    # Create a dictionary from the acronyms dataframe
    acronyms_dict = pd.Series(acronyms_df['Meaning'].values, index=acronyms_df['Acronyms']).to_dict()

    # Function to replace acronyms in a string using word boundaries
    def replace_in_string(text):
        if isinstance(text, str):  # Ensure the text is a string
            for acronym, meaning in acronyms_dict.items():
                # Use regex to replace acronym with meaning, ensuring word boundaries
                text = re.sub(rf'\b{re.escape(acronym)}\b', meaning, text)
        return text

    # Apply the function to the specified column
    df[column_name] = df[column_name].apply(replace_in_string)
    
# Function to add acronym meanings, keeping the original acronym
# def add_acronym_meanings(df, column_name, acronyms_df):
#     # Create a dictionary from the acronyms dataframe
#     acronyms_dict = pd.Series(acronyms_df['Meaning'].values, index=acronyms_df['Acronyms']).to_dict()

#     # Define a function to replace acronyms in a string
#     def replace_in_string(text):
#         words = text.split()
#         replaced_text = ' '.join([f"{word} [{acronyms_dict[word]}]" if word in acronyms_dict else word for word in words])
#         return replaced_text

#     # Apply the function to the specified column
#     df[column_name] = df[column_name].apply(replace_in_string)
def add_acronym_meanings(df, column_name, acronyms_df):
    # Create a dictionary from the acronyms dataframe
    acronyms_dict = pd.Series(acronyms_df['Meaning'].values, index=acronyms_df['Acronyms']).to_dict()

    # Function to replace acronyms with acronym and meaning in brackets using word boundaries
    def replace_in_string(text):
        if isinstance(text, str):  # Ensure the text is a string
            for acronym, meaning in acronyms_dict.items():
                # Use regex to find acronym and replace with acronym [meaning]
                text = re.sub(rf'\b{re.escape(acronym)}\b', f"{acronym} [{meaning}]", text)
        return text

    # Apply the function to the specified column
    df[column_name] = df[column_name].apply(replace_in_string)

In [36]:
# SCORES BEFORE PREPROCESSING
zeroshot_bhc_scores = pd.read_csv("/work/AI-EHR/discharge_me/data/Experiments/zeroshot-llama8b_instruct/troy_scoring_experiment/Zeroshot_BHC_scores_df.csv")

# best hadm_ids -> using for scoring
hadm_ids_to_keep = [22007257, 22289537, 22988214, 25498079, 22887667]

filtered_df = zeroshot_bhc_scores[zeroshot_bhc_scores['hadm_id'].isin(hadm_ids_to_keep)]

filtered_df

,hadm_id,BHC_LLM_out,BHC_human_generated,meteor_score,bertscore_precision,bertscore_recall,bertscore_f1,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores
50,22007257,Brief Hospital Course\nThe patient with a hist...,Hospital course: Ms. ___ is a very pleasant __...,0.294944,0.777150,0.757955,0.767433,0.098176,0.359649,0.150442,0.210526,0.242948
143,22289537,Brief Hospital Course\nThe patient with a hist...,"Ms. ___ is a ___ female with a hx of CAD, HTN,...",0.253854,0.796746,0.767335,0.781764,0.000007,0.411523,0.091286,0.246914,0.290196
192,22988214,Brief Hospital Course\nThe patient with a hist...,Mr. ___ is a ___ man with a past medical histo...,0.287684,0.784256,0.776676,0.780448,0.056181,0.336508,0.102236,0.196825,0.273612
225,25498079,Brief Hospital Course\nThe patient with a hist...,Assessment and plan: ___ year old male without...,0.222052,0.803499,0.767821,0.785255,0.054607,0.347826,0.103586,0.197628,0.300813
234,22887667,Brief Hospital Course\nThe patient with a hist...,An ___ year old female with PMH nephrolithiasi...,0.235656,0.769507,0.786502,0.777912,0.039366,0.351254,0.057762,0.200717,0.306229


In [76]:
filtered_df.describe()

,hadm_id,meteor_score,bertscore_precision,bertscore_recall,bertscore_f1,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores
count,5.000000e+00,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,2.313415e+07,0.258838,0.786232,0.771258,0.778562,0.049668,0.361352,0.101063,0.210522,0.282760
std,1.383245e+06,0.031824,0.013918,0.010792,0.006762,0.035328,0.029249,0.033229,0.021061,0.025498
min,2.200726e+07,0.222052,0.769507,0.757955,0.767433,0.000007,0.336508,0.057762,0.196825,0.242948
25%,2.228954e+07,0.235656,0.777150,0.767335,0.777912,0.039366,0.347826,0.091286,0.197628,0.273612
50%,2.288767e+07,0.253854,0.784256,0.767821,0.780448,0.054607,0.351254,0.102236,0.200717,0.290196
75%,2.298821e+07,0.287684,0.796746,0.776676,0.781764,0.056181,0.359649,0.103586,0.210526,0.300813
max,2.549808e+07,0.294944,0.803499,0.786502,0.785255,0.098176,0.411523,0.150442,0.246914,0.306229


In [77]:
original_dataframe = pd.DataFrame()
original_dataframe['refs'] = filtered_df['BHC_human_generated']
original_dataframe['hyps'] = filtered_df['BHC_LLM_out']

original_dataframe

,refs,hyps
50,Hospital course: Ms. ___ is a very pleasant __...,Brief Hospital Course\nThe patient with a hist...
143,"Ms. ___ is a ___ female with a hx of CAD, HTN,...",Brief Hospital Course\nThe patient with a hist...
192,Mr. ___ is a ___ man with a past medical histo...,Brief Hospital Course\nThe patient with a hist...
225,Assessment and plan: ___ year old male without...,Brief Hospital Course\nThe patient with a hist...
234,An ___ year old female with PMH nephrolithiasi...,Brief Hospital Course\nThe patient with a hist...


In [6]:
pip install numpy==1.22.0

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 16.8 MB 3.8 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: Could not install packages due to an OSError: [Errno 16] Device or resource busy: '.nfs34faa6642777302a000167b4'

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install rouge_score

Defaulting to user installation because normal site-packages is not writeable
  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=dcffedb33d0343a26ae2556b1226eefc6aa715686ce7261dcad60105edc54537
  Stored in directory: /home/tr.caron/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [29]:
from bertscore import BertScore
# from align import AlignScorer
from bleu import Bleu
from rouge import Rouge, Rouge1, Rouge2, RougeL

bert_scorer = BertScore()
#align_scorer = AlignScorer()
bleu_scorer = Bleu(n=4)
rouge1_scorer = Rouge1()
rouge2_scorer = Rouge2()
rougeL_scorer = RougeL()

/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/tr.caron/.local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Removing hashtags

In [14]:
no_hashtags_dataframe = original_dataframe.copy()
remove_hashtags(no_hashtags_dataframe, "refs")
remove_hashtags(no_hashtags_dataframe, "hyps")

In [15]:
refs = no_hashtags_dataframe['refs'].tolist()
hyps = no_hashtags_dataframe['hyps'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

no_hashtags_dataframe['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
no_hashtags_dataframe['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
no_hashtags_dataframe['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
no_hashtags_dataframe['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
no_hashtags_dataframe['task-organizers-bertscores'] = bertscores

# Calculate METEOR score for each row
for index, row in no_hashtags_dataframe.iterrows():
    prediction = [row['hyps']]
    reference = [row['refs']]
    result = meteor.compute(predictions=prediction, references=reference)
    no_hashtags_dataframe.at[index, 'meteor_score'] = result['meteor']

In [14]:
no_hashtags_dataframe.describe()

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.050804,0.361352,0.101063,0.210522,0.279160,0.257082
std,0.035900,0.029249,0.033229,0.021061,0.025713,0.033329
min,0.000007,0.336508,0.057762,0.196825,0.241376,0.218346
25%,0.039358,0.347826,0.091286,0.197628,0.266218,0.231963
50%,0.054481,0.351254,0.102236,0.200717,0.285372,0.253700
75%,0.061175,0.359649,0.103586,0.210526,0.298229,0.286229
max,0.098998,0.411523,0.150442,0.246914,0.304604,0.295171


## Removing semicolons

In [16]:
no_semicolons_dataframe = original_dataframe.copy()
remove_semicolons(no_semicolons_dataframe, "refs")
remove_semicolons(no_semicolons_dataframe, "hyps")

In [17]:
refs = no_semicolons_dataframe['refs'].tolist()
hyps = no_semicolons_dataframe['hyps'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

no_semicolons_dataframe['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
no_semicolons_dataframe['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
no_semicolons_dataframe['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
no_semicolons_dataframe['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
no_semicolons_dataframe['task-organizers-bertscores'] = bertscores

# Calculate METEOR score for each row
for index, row in no_semicolons_dataframe.iterrows():
    prediction = [row['hyps']]
    reference = [row['refs']]
    result = meteor.compute(predictions=prediction, references=reference)
    no_semicolons_dataframe.at[index, 'meteor_score'] = result['meteor']

In [18]:
no_hashtags_dataframe.describe()

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.050804,0.361352,0.101063,0.210522,0.279160,0.257082
std,0.035900,0.029249,0.033229,0.021061,0.025713,0.033329
min,0.000007,0.336508,0.057762,0.196825,0.241376,0.218346
25%,0.039358,0.347826,0.091286,0.197628,0.266218,0.231963
50%,0.054481,0.351254,0.102236,0.200717,0.285372,0.253700
75%,0.061175,0.359649,0.103586,0.210526,0.298229,0.286229
max,0.098998,0.411523,0.150442,0.246914,0.304604,0.295171


## Removing consecutive equals signs

In [19]:
no_consecutive_equals_dataframe = original_dataframe.copy()
remove_multiple_equals(no_consecutive_equals_dataframe, "refs")
remove_multiple_equals(no_consecutive_equals_dataframe, "hyps")

In [20]:
refs = no_consecutive_equals_dataframe['refs'].tolist()
hyps = no_consecutive_equals_dataframe['hyps'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

no_consecutive_equals_dataframe['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
no_consecutive_equals_dataframe['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
no_consecutive_equals_dataframe['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
no_consecutive_equals_dataframe['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
no_consecutive_equals_dataframe['task-organizers-bertscores'] = bertscores

# Calculate METEOR score for each row
for index, row in no_consecutive_equals_dataframe.iterrows():
    prediction = [row['hyps']]
    reference = [row['refs']]
    result = meteor.compute(predictions=prediction, references=reference)
    no_consecutive_equals_dataframe.at[index, 'meteor_score'] = result['meteor']

In [21]:
no_consecutive_equals_dataframe.describe()

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.049668,0.361352,0.101063,0.210522,0.282760,0.258838
std,0.035328,0.029249,0.033229,0.021061,0.025498,0.031824
min,0.000007,0.336508,0.057762,0.196825,0.242948,0.222052
25%,0.039366,0.347826,0.091286,0.197628,0.273612,0.235656
50%,0.054607,0.351254,0.102236,0.200717,0.290196,0.253854
75%,0.056181,0.359649,0.103586,0.210526,0.300813,0.287684
max,0.098176,0.411523,0.150442,0.246914,0.306229,0.294944


## REPLACING acronyms with their meaning

In [79]:
#acronyms_df = table[0]

acronyms_replaced_dataframe = original_dataframe.copy()
replace_acronyms(acronyms_replaced_dataframe, "refs", acronyms_df)
replace_acronyms(acronyms_replaced_dataframe, "hyps", acronyms_df)

In [80]:
refs = acronyms_replaced_dataframe['refs'].tolist()
hyps = acronyms_replaced_dataframe['hyps'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

acronyms_replaced_dataframe['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
acronyms_replaced_dataframe['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
acronyms_replaced_dataframe['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
acronyms_replaced_dataframe['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
acronyms_replaced_dataframe['task-organizers-bertscores'] = bertscores

# Calculate METEOR score for each row
for index, row in acronyms_replaced_dataframe.iterrows():
    prediction = [row['hyps']]
    reference = [row['refs']]
    result = meteor.compute(predictions=prediction, references=reference)
    acronyms_replaced_dataframe.at[index, 'meteor_score'] = result['meteor']

In [81]:
acronyms_replaced_dataframe.describe()

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.073731,0.382053,0.122073,0.228260,0.309980,0.284118
std,0.032949,0.052988,0.045506,0.045285,0.032214,0.031740
min,0.039882,0.326154,0.068966,0.190769,0.264219,0.232773
25%,0.054026,0.347490,0.099071,0.193050,0.301877,0.278738
50%,0.056439,0.363014,0.108949,0.205479,0.303881,0.289541
75%,0.109097,0.420168,0.146939,0.260504,0.331126,0.306678
max,0.109212,0.453441,0.186441,0.291498,0.348798,0.312860


## Putting acronyms with their meaning next to them in square brackets

In [82]:
#acronyms_df = table[0]

acronyms_next_to_meanings_dataframe = original_dataframe.copy()
add_acronym_meanings(acronyms_next_to_meanings_dataframe, "refs", acronyms_df)
add_acronym_meanings(acronyms_next_to_meanings_dataframe, "hyps", acronyms_df)

In [83]:
refs = acronyms_next_to_meanings_dataframe['refs'].tolist()
hyps = acronyms_next_to_meanings_dataframe['hyps'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

acronyms_next_to_meanings_dataframe['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
acronyms_next_to_meanings_dataframe['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
acronyms_next_to_meanings_dataframe['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
acronyms_next_to_meanings_dataframe['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
acronyms_next_to_meanings_dataframe['task-organizers-bertscores'] = bertscores

# Calculate METEOR score for each row
for index, row in acronyms_next_to_meanings_dataframe.iterrows():
    prediction = [row['hyps']]
    reference = [row['refs']]
    result = meteor.compute(predictions=prediction, references=reference)
    acronyms_next_to_meanings_dataframe.at[index, 'meteor_score'] = result['meteor']

In [48]:
acronyms_next_to_meanings_dataframe.to_csv("acronym_meanings_added_in_brackets.csv",index=False)

In [84]:
acronyms_next_to_meanings_dataframe.describe()

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.051647,0.381123,0.123637,0.227325,0.314476,0.287660
std,0.035828,0.049677,0.039827,0.042544,0.029213,0.030894
min,0.000007,0.329341,0.080000,0.189394,0.272276,0.238100
25%,0.044009,0.348485,0.108434,0.191617,0.304126,0.284125
50%,0.053510,0.364238,0.114504,0.211921,0.312602,0.290510
75%,0.060886,0.412955,0.127490,0.259109,0.337151,0.306509
max,0.099822,0.450593,0.187755,0.284585,0.346225,0.319057


## Trying removing consecutive equals on full 250 hadm_ids

In [50]:
zeroshot_bhc_scores.describe()

,hadm_id,meteor_score,bertscore_precision,bertscore_recall,bertscore_f1,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000,2.500000e+02,250.000000,250.000000,250.000000,250.000000
mean,2.498990e+07,0.228323,0.781773,0.775893,0.778593,3.292901e-02,0.313158,0.084421,0.187212,0.280098
std,2.727081e+06,0.056019,0.027242,0.026346,0.023233,4.418558e-02,0.068705,0.048726,0.049383,0.064906
min,2.006350e+07,0.089823,0.694748,0.692853,0.699865,1.881899e-14,0.103560,0.000000,0.064171,0.093414
25%,2.271984e+07,0.190075,0.764479,0.762092,0.765064,8.316304e-10,0.267873,0.049844,0.151071,0.238340
50%,2.482564e+07,0.223672,0.781092,0.775661,0.778624,1.016631e-02,0.317325,0.080123,0.184710,0.274941
75%,2.711815e+07,0.258857,0.801148,0.792880,0.795442,5.238075e-02,0.353456,0.106464,0.215281,0.325899
max,2.999192e+07,0.392270,0.843972,0.856289,0.842104,2.477786e-01,0.478964,0.273171,0.357488,0.454456


In [51]:
no_consecutive_equals_full_dataframe = pd.DataFrame()
no_consecutive_equals_full_dataframe['refs'] = zeroshot_bhc_scores['BHC_human_generated']
no_consecutive_equals_full_dataframe['hyps'] = zeroshot_bhc_scores['BHC_LLM_out']

remove_multiple_equals(no_consecutive_equals_full_dataframe, "refs")
remove_multiple_equals(no_consecutive_equals_full_dataframe, "hyps")

In [52]:
refs = no_consecutive_equals_full_dataframe['refs'].tolist()
hyps = no_consecutive_equals_full_dataframe['hyps'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

no_consecutive_equals_full_dataframe['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
no_consecutive_equals_full_dataframe['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
no_consecutive_equals_full_dataframe['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
no_consecutive_equals_full_dataframe['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
no_consecutive_equals_full_dataframe['task-organizers-bertscores'] = bertscores

# Calculate METEOR score for each row
for index, row in no_consecutive_equals_full_dataframe.iterrows():
    prediction = [row['hyps']]
    reference = [row['refs']]
    result = meteor.compute(predictions=prediction, references=reference)
    no_consecutive_equals_full_dataframe.at[index, 'meteor_score'] = result['meteor']

In [53]:
no_consecutive_equals_full_dataframe.describe()

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000,250.000000
mean,3.294479e-02,0.313158,0.084421,0.187212,0.282973,0.228493
std,4.421452e-02,0.068705,0.048726,0.049383,0.063540,0.056037
min,1.881899e-14,0.103560,0.000000,0.064171,0.109364,0.089823
25%,8.316304e-10,0.267873,0.049844,0.151071,0.239412,0.190075
50%,1.016631e-02,0.317325,0.080123,0.184710,0.279993,0.223672
75%,5.238075e-02,0.353456,0.106464,0.215281,0.325899,0.260466
max,2.477786e-01,0.478964,0.273171,0.357488,0.454456,0.394648


## Trying removing semicolons on full 250 hadm_ids

In [54]:
no_semicolons_full_dataframe = pd.DataFrame()
no_semicolons_full_dataframe['refs'] = zeroshot_bhc_scores['BHC_human_generated']
no_semicolons_full_dataframe['hyps'] = zeroshot_bhc_scores['BHC_LLM_out']

remove_semicolons(no_semicolons_full_dataframe, "refs")
remove_semicolons(no_semicolons_full_dataframe, "hyps")

In [55]:
refs = no_semicolons_full_dataframe['refs'].tolist()
hyps = no_semicolons_full_dataframe['hyps'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

no_semicolons_full_dataframe['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
no_semicolons_full_dataframe['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
no_semicolons_full_dataframe['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
no_semicolons_full_dataframe['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
no_semicolons_full_dataframe['task-organizers-bertscores'] = bertscores

# Calculate METEOR score for each row
for index, row in no_semicolons_full_dataframe.iterrows():
    prediction = [row['hyps']]
    reference = [row['refs']]
    result = meteor.compute(predictions=prediction, references=reference)
    no_semicolons_full_dataframe.at[index, 'meteor_score'] = result['meteor']

In [56]:
no_semicolons_full_dataframe.describe()

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000,250.000000
mean,3.292901e-02,0.313158,0.084421,0.187212,0.280061,0.228387
std,4.418558e-02,0.068705,0.048726,0.049383,0.064939,0.055997
min,1.881899e-14,0.103560,0.000000,0.064171,0.093224,0.089823
25%,8.316304e-10,0.267873,0.049844,0.151071,0.238340,0.190075
50%,1.016631e-02,0.317325,0.080123,0.184710,0.275008,0.223672
75%,5.238075e-02,0.353456,0.106464,0.215281,0.325899,0.258857
max,2.477786e-01,0.478964,0.273171,0.357488,0.454456,0.392270


## Combination of removing consecutive equal signs and adding acronym meanings

In [85]:
#acronyms_df = table[0]

acronym_meanings_and_no_equals_dataframe = no_consecutive_equals_full_dataframe.copy()
add_acronym_meanings(acronym_meanings_and_no_equals_dataframe, "refs", acronyms_df)
add_acronym_meanings(acronym_meanings_and_no_equals_dataframe, "hyps", acronyms_df)

In [86]:
refs = acronym_meanings_and_no_equals_dataframe['refs'].tolist()
hyps = acronym_meanings_and_no_equals_dataframe['hyps'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

acronym_meanings_and_no_equals_dataframe['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
acronym_meanings_and_no_equals_dataframe['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
acronym_meanings_and_no_equals_dataframe['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
acronym_meanings_and_no_equals_dataframe['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
acronym_meanings_and_no_equals_dataframe['task-organizers-bertscores'] = bertscores

# Calculate METEOR score for each row
for index, row in acronym_meanings_and_no_equals_dataframe.iterrows():
    prediction = [row['hyps']]
    reference = [row['refs']]
    result = meteor.compute(predictions=prediction, references=reference)
    acronym_meanings_and_no_equals_dataframe.at[index, 'meteor_score'] = result['meteor']

In [87]:
acronym_meanings_and_no_equals_dataframe.describe()

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
count,2.500000e+02,250.000000,250.000000,250.000000,250.000000,250.000000
mean,4.006671e-02,0.317221,0.095054,0.190458,0.296970,0.240801
std,4.726326e-02,0.071230,0.054041,0.052739,0.068570,0.064779
min,1.394145e-14,0.100946,0.000000,0.081081,0.132296,0.077361
25%,4.003037e-06,0.272263,0.056284,0.152819,0.247420,0.192981
50%,3.364121e-02,0.317774,0.083582,0.184744,0.295817,0.232515
75%,6.048472e-02,0.360621,0.123653,0.219839,0.345940,0.284074
max,2.398562e-01,0.493590,0.274882,0.366197,0.470084,0.434687


## Percent change from baseline

In [88]:
# making baseline scores same format
baseline_scores = original_dataframe.copy()

refs = baseline_scores['refs'].tolist()
hyps = baseline_scores['hyps'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

baseline_scores['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
baseline_scores['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
baseline_scores['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
baseline_scores['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
baseline_scores['task-organizers-bertscores'] = bertscores

# Calculate METEOR score for each row
for index, row in baseline_scores.iterrows():
    prediction = [row['hyps']]
    reference = [row['refs']]
    result = meteor.compute(predictions=prediction, references=reference)
    baseline_scores.at[index, 'meteor_score'] = result['meteor']


In [89]:
baseline_scores.describe()

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.049668,0.361352,0.101063,0.210522,0.282760,0.258838
std,0.035328,0.029249,0.033229,0.021061,0.025498,0.031824
min,0.000007,0.336508,0.057762,0.196825,0.242948,0.222052
25%,0.039366,0.347826,0.091286,0.197628,0.273612,0.235656
50%,0.054607,0.351254,0.102236,0.200717,0.290196,0.253854
75%,0.056181,0.359649,0.103586,0.210526,0.300813,0.287684
max,0.098176,0.411523,0.150442,0.246914,0.306229,0.294944


In [90]:
combined_means_df_5 = pd.DataFrame([baseline_scores.mean(), no_hashtags_dataframe.mean(), no_semicolons_dataframe.mean(), no_consecutive_equals_dataframe.mean(), acronyms_replaced_dataframe.mean(), acronyms_next_to_meanings_dataframe.mean()])
combined_means_df_5.index = ['Baseline Scores (5)', 'Remove "#" (5)', 'Remove ";" (5)', 'Remove "==" (5)', 'Replaced acronyms (5)', 'Meaning added in [] (5)']
combined_means_df_5

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
Baseline Scores (5),0.049668,0.361352,0.101063,0.210522,0.282760,0.258838
"Remove ""#"" (5)",0.050804,0.361352,0.101063,0.210522,0.279160,0.257082
"Remove "";"" (5)",0.049668,0.361352,0.101063,0.210522,0.283139,0.259250
"Remove ""=="" (5)",0.049668,0.361352,0.101063,0.210522,0.282760,0.258838
Replaced acronyms (5),0.073731,0.382053,0.122073,0.228260,0.309980,0.284118
Meaning added in [] (5),0.051647,0.381123,0.123637,0.227325,0.314476,0.287660


In [91]:
# Calculate percent change from baseline for each row
baseline = combined_means_df_5.loc['Baseline Scores (5)']
df_percent_change = combined_means_df_5.apply(lambda row: (row - baseline) / baseline * 100, axis=1)

# Keep the baseline row as is
df_percent_change.loc['Baseline Scores (5)'] = baseline

# Display the modified dataframe
df_percent_change

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
Baseline Scores (5),0.049668,0.361352,0.101063,0.210522,0.282760,0.258838
"Remove ""#"" (5)",2.287593,0.000000,0.000000,0.000000,-1.273020,-0.678486
"Remove "";"" (5)",0.000000,0.000000,0.000000,0.000000,0.134262,0.159122
"Remove ""=="" (5)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Replaced acronyms (5),48.449091,5.728873,20.789705,8.425761,9.626669,9.766570
Meaning added in [] (5),3.984590,5.471266,22.336688,7.981577,11.216653,11.135100


In [65]:
# making baseline scores same format
baseline_scores_250 = pd.DataFrame()
baseline_scores_250['refs'] = zeroshot_bhc_scores['BHC_human_generated']
baseline_scores_250['hyps'] = zeroshot_bhc_scores['BHC_LLM_out']

refs = baseline_scores_250['refs'].tolist()
hyps = baseline_scores_250['hyps'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

baseline_scores_250['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
baseline_scores_250['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
baseline_scores_250['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
baseline_scores_250['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
baseline_scores_250['task-organizers-bertscores'] = bertscores

# Calculate METEOR score for each row
for index, row in baseline_scores_250.iterrows():
    prediction = [row['hyps']]
    reference = [row['refs']]
    result = meteor.compute(predictions=prediction, references=reference)
    baseline_scores_250.at[index, 'meteor_score'] = result['meteor']


In [92]:
combined_means_df_250 = pd.DataFrame([baseline_scores_250.mean(), no_consecutive_equals_full_dataframe.mean(), no_semicolons_full_dataframe.mean(), acronym_meanings_and_no_equals_dataframe.mean()])
combined_means_df_250.index = ['Baseline Scores (250)', 'Remove "==" (250)', 'Remove ";" (250)', 'Meanings added in [] and removed "==" (250)']
combined_means_df_250

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
Baseline Scores (250),0.032929,0.313158,0.084421,0.187212,0.280098,0.228323
"Remove ""=="" (250)",0.032945,0.313158,0.084421,0.187212,0.282973,0.228493
"Remove "";"" (250)",0.032929,0.313158,0.084421,0.187212,0.280061,0.228387
"Meanings added in [] and removed ""=="" (250)",0.040067,0.317221,0.095054,0.190458,0.296970,0.240801


In [93]:
# Calculate percent change from baseline for each row
baseline250 = combined_means_df_250.loc['Baseline Scores (250)']
df_percent_change_250 = combined_means_df_250.apply(lambda row: (row - baseline250) / baseline250 * 100, axis=1)

# Keep the baseline row as is
df_percent_change_250.loc['Baseline Scores (250)'] = baseline250

# Display the modified dataframe
df_percent_change_250

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
Baseline Scores (250),3.292901e-02,0.313158,0.084421,0.187212,0.280098,0.228323
"Remove ""=="" (250)",4.792190e-02,0.000000,0.000000,0.000000,1.026440,0.074044
"Remove "";"" (250)",5.293357e-11,0.000000,0.000000,0.000000,-0.013178,0.027953
"Meanings added in [] and removed ""=="" (250)",2.167600e+01,1.297647,12.595235,1.734200,6.023574,5.465061


In [94]:
def count_hashtags(df, column_name):
    return df[column_name].str.count('#').sum()

def count_semicolons(df, column_name):
    return df[column_name].str.count(';').sum()

def count_equal_signs(df, column_name):
    pattern = r'={2,}'  # regex to match strings of two or more equal signs
    return df[column_name].apply(lambda x: len(re.findall(pattern, str(x)))).sum()

def count_total_acronyms(acronym_table, df, column_name):
    total_count = 0

    # loop through each acronym in acronym table
    for acronym in acronym_table['Acronyms']:
        # Convert acronym to string in case of NaN or other types
        acronym = str(acronym)
        
        # count occurrences using word boundaries
        total_count += df[column_name].str.count(r'\b' + acronym + r'\b').sum()
    
    return total_count

In [95]:
df_percent_change

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
Baseline Scores (5),0.049668,0.361352,0.101063,0.210522,0.282760,0.258838
"Remove ""#"" (5)",2.287593,0.000000,0.000000,0.000000,-1.273020,-0.678486
"Remove "";"" (5)",0.000000,0.000000,0.000000,0.000000,0.134262,0.159122
"Remove ""=="" (5)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Replaced acronyms (5),48.449091,5.728873,20.789705,8.425761,9.626669,9.766570
Meaning added in [] (5),3.984590,5.471266,22.336688,7.981577,11.216653,11.135100


In [96]:
#acronyms_df = table[0]

df_percent_change['Number of changes made'] = [
    "N/A",  # Baseline, no preprocessing applied
    count_hashtags(original_dataframe, "refs") + count_hashtags(original_dataframe, "hyps"),
    count_semicolons(original_dataframe, "refs") + count_semicolons(original_dataframe, "hyps"),
    count_equal_signs(original_dataframe, "refs") + count_equal_signs(original_dataframe, "hyps"),
    count_total_acronyms(acronyms_df, original_dataframe, "refs") + count_total_acronyms(acronyms_df, original_dataframe, "hyps"),
    count_total_acronyms(acronyms_df, original_dataframe, "refs") + count_total_acronyms(acronyms_df, original_dataframe, "hyps")
]

In [97]:
df_percent_change

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score,Number of changes made
Baseline Scores (5),0.049668,0.361352,0.101063,0.210522,0.282760,0.258838,N/A
"Remove ""#"" (5)",2.287593,0.000000,0.000000,0.000000,-1.273020,-0.678486,17
"Remove "";"" (5)",0.000000,0.000000,0.000000,0.000000,0.134262,0.159122,1
"Remove ""=="" (5)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
Replaced acronyms (5),48.449091,5.728873,20.789705,8.425761,9.626669,9.766570,62
Meaning added in [] (5),3.984590,5.471266,22.336688,7.981577,11.216653,11.135100,62


In [98]:
df_percent_change_250['Number of changes made'] = [
    "N/A",  # Baseline, no preprocessing applied
    count_equal_signs(baseline_scores_250, "refs") + count_equal_signs(baseline_scores_250, "hyps"),
    count_semicolons(baseline_scores_250, "refs") + count_semicolons(baseline_scores_250, "hyps"),
    count_equal_signs(baseline_scores_250, "refs") + count_equal_signs(baseline_scores_250, "hyps") +
    count_total_acronyms(acronyms_df, baseline_scores_250, "refs") + count_total_acronyms(acronyms_df, baseline_scores_250, "hyps")
]

In [99]:
df_percent_change_250

,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score,Number of changes made
Baseline Scores (250),3.292901e-02,0.313158,0.084421,0.187212,0.280098,0.228323,N/A
"Remove ""=="" (250)",4.792190e-02,0.000000,0.000000,0.000000,1.026440,0.074044,32
"Remove "";"" (250)",5.293357e-11,0.000000,0.000000,0.000000,-0.013178,0.027953,15
"Meanings added in [] and removed ""=="" (250)",2.167600e+01,1.297647,12.595235,1.734200,6.023574,5.465061,3150


## Scoring context-adjusted results from troyTest

In [12]:
import pandas as pd

In [14]:
pwd

'/work/AI-EHR/discharge_me/data/Experiments/zeroshot-llama8b_instruct/troy_scoring_experiment'

In [16]:
df = pd.read_csv("../../../troyTest/results_bhc_smwtest.csv")

In [37]:
df['refs'] = zeroshot_bhc_scores['BHC_human_generated']

In [41]:
refs = df['refs'].tolist()
hyps = df['BHC_LLM'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

df['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
df['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
df['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
df['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
df['task-organizers-bertscores'] = bertscores

# Calculate METEOR score for each row
for index, row in df.iterrows():
    prediction = [row['BHC_LLM']]
    reference = [row['refs']]
    result = meteor.compute(predictions=prediction, references=reference)
    df.at[index, 'meteor_score'] = result['meteor']

In [43]:
df.describe()

,hadm_id,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
count,2.500000e+02,2.500000e+02,250.000000,250.000000,250.000000,250.000000,250.000000
mean,2.498990e+07,3.343273e-02,0.308755,0.087393,0.187349,0.275310,0.237722
std,2.727081e+06,4.286420e-02,0.067068,0.051564,0.050888,0.065654,0.059504
min,2.006350e+07,2.659249e-14,0.084788,0.000000,0.059850,0.058180,0.109232
25%,2.271984e+07,2.428186e-06,0.264121,0.048382,0.151631,0.231998,0.196716
50%,2.482564e+07,2.030136e-02,0.308317,0.080550,0.182482,0.277605,0.227428
75%,2.711815e+07,5.701531e-02,0.352245,0.109362,0.215336,0.317579,0.271195
max,2.999192e+07,1.993082e-01,0.482456,0.260223,0.346667,0.452648,0.454622


In [44]:
print(df.iloc[0]['refs'])


Ms. ___ is an ___ year old woman with a history of DM, HTN, 
HLD, TIA's, temporal arteritis, and 3 vessel CAD with multiple 
PCI's and stents, PVD who presents from Dr. ___ office with 
chest pain and concern for unstable angina. 

# CORONARIES:  History of multiple PCI's, now with symptoms 
concerning for unstable angina over past ___ weeks. She was 
taken to the cath lab which did not show any evidence of new 
coronary disease, no intervention was made. Continued home ASA, 
plavix, BB. We increased the dose of valsartan as her BPs were 
running high in the hospital.


In [45]:
print(df.iloc[0]['BHC_LLM'])


Brief Hospital Course
The patient with a history of diabetes mellitus, hypertension, hyperlipidemia, transient ischemic attacks, temporal arteritis, and 3-vessel coronary artery disease with multiple percutaneous coronary interventions and stents, peripheral vascular disease presents with chest pain and concern for unstable angina. She was admitted to the cardiology floor for cardiac catheterization and evaluation of her symptoms.

The patient's symptoms included intermittent left back pain for three weeks, which progressed to substernal chest pain with radiation to the left arm, pleuritic, and somewhat relieved by nitroglycerin. She also experienced mild shortness of breath, nausea, and no diaphoresis.

Problem-based hospital course---
# Chest pain --- The patient presented with chest pain and concern for unstable angina.
The diagnostic work up included cardiac catheterization, which revealed no angiographically-apparent flow-limiting stenoses. The left main coronary artery had a 10% 

## Scoring chain-of-thought results from troyTest

In [2]:
import pandas as pd

bhc_df = pd.read_csv("../../../troyTest/5_CoT_bhc_results.csv")
#di_df = pd.read_csv("../../../troyTest/CoT_results_di.csv")

In [6]:
bhc_df

,hadm_id,BHC_LLM,BHC_human
0,22007257,Ms. ___ was admitted to the hospital due to ch...,2162 Brief Hospital Course:\nHospital cours...
1,22289537,Ms. ___ was admitted to the hospital due to a ...,9265 Brief Hospital Course:\nMs. ___ is a _...
2,22988214,The patient has a relevant past medical histor...,1373 Brief Hospital Course:\nMr. ___ is a _...
3,25498079,The patient was admitted to the hospital due t...,2144 Brief Hospital Course:\nAssessment and...
4,22887667,The patient has a relevant past medical histor...,2485 Brief Hospital Course:\nAn ___ year ol...


In [ ]:
zeroshot_bhc_scores = pd.read_csv("/work/AI-EHR/discharge_me/data/Experiments/zeroshot-llama8b_instruct/troy_scoring_experiment/Zeroshot_BHC_scores_df.csv")

# best hadm_ids -> using for scoring
hadm_ids_to_keep = [22007257, 22289537, 22988214, 25498079, 22887667]

filtered_df = zeroshot_bhc_scores[zeroshot_bhc_scores['hadm_id'].isin(hadm_ids_to_keep)]

# Merge based on 'hadm_id'
bhc_df = pd.merge(bhc_df, filtered_df[['hadm_id', 'BHC_human_generated']], on='hadm_id', how='left')

MergeError: Passing 'suffixes' which cause duplicate columns {'BHC_human_generated_x'} is not allowed.

In [28]:
bhc_df

,hadm_id,BHC_LLM,BHC_human_generated
0,22007257,Ms. ___ was admitted to the hospital due to ch...,Hospital course: Ms. ___ is a very pleasant __...
1,22289537,Ms. ___ was admitted to the hospital due to a ...,"Ms. ___ is a ___ female with a hx of CAD, HTN,..."
2,22988214,The patient has a relevant past medical histor...,Mr. ___ is a ___ man with a past medical histo...
3,25498079,The patient was admitted to the hospital due t...,Assessment and plan: ___ year old male without...
4,22887667,The patient has a relevant past medical histor...,An ___ year old female with PMH nephrolithiasi...


In [33]:
refs = bhc_df['BHC_human_generated'].tolist()
hyps = bhc_df['BHC_LLM'].tolist()

bleuscores = bleu_scorer(refs, hyps)
rouge1scores = rouge1_scorer(refs, hyps)
rouge2scores = rouge2_scorer(refs, hyps)
rougeLscores = rougeL_scorer(refs, hyps)
bertscores = bert_scorer(refs, hyps)

bhc_df['task-organizers-bleuscores'] = bleuscores
# rouge returns 1 key-value pair so take the value
bhc_df['task-organizers-rouge1scores'] = list(rouge1scores.values())[0]
bhc_df['task-organizers-rouge2scores'] = list(rouge2scores.values())[0]
bhc_df['task-organizers-rougeLscores'] = list(rougeLscores.values())[0]
bhc_df['task-organizers-bertscores'] = bertscores

# Calculate METEOR score for each row
for index, row in bhc_df.iterrows():
    prediction = [row['BHC_LLM']]
    reference = [row['BHC_human_generated']]
    result = meteor.compute(predictions=prediction, references=reference)
    bhc_df.at[index, 'meteor_score'] = result['meteor']

In [36]:
bhc_df.describe()

,hadm_id,task-organizers-bleuscores,task-organizers-rouge1scores,task-organizers-rouge2scores,task-organizers-rougeLscores,task-organizers-bertscores,meteor_score
count,5.000000e+00,5.000000e+00,5.000000,5.000000,5.000000,5.000000,5.000000
mean,2.313415e+07,7.500799e-03,0.153854,0.035166,0.087915,0.254668,0.215380
std,1.383245e+06,9.526185e-03,0.055421,0.008237,0.024226,0.042223,0.043088
min,2.200726e+07,2.946862e-10,0.080863,0.024825,0.052830,0.201315,0.170858
25%,2.228954e+07,1.471043e-06,0.109375,0.029872,0.072443,0.231598,0.182978
50%,2.288767e+07,7.003296e-03,0.177156,0.035047,0.102564,0.250408,0.214681
75%,2.298821e+07,7.182547e-03,0.199697,0.040562,0.102874,0.279524,0.227913
max,2.549808e+07,2.331668e-02,0.202177,0.045524,0.108865,0.310492,0.280470


In [13]:
print(di_df.iloc[0]['discharge_instructions'])

Dear [Patient's Name],

I am writing to inform you about your recent hospital admission and the treatments you received during your stay. You were admitted to the hospital due to an abnormal stress test, which revealed evidence of inducible ischemia and severe aortic insufficiency. The symptoms and medical condition that caused you to come to the hospital were the abnormal stress test results, which indicated a severe aortic insufficiency and inducible ischemia, requiring further evaluation and management.

During your hospital stay, you received treatment for your aortic regurgitation, aortic stenosis, coronary artery disease, inducible ischemia, and severe aortic insufficiency. You were prescribed Lisinopril 5 mg PO DAILY for each of these conditions.

As part of your treatment, we also stopped or continued some of your previous medications. The medications that were stopped or continued but not present in the admission list include Diltiazem Extended-Release 120 mg PO DAILY, Glutami